In [17]:
# Import necessary libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

#Load the data from CSV files
test_data_features = pd.read_csv('test_data_features.csv')
training_data_features = pd.read_csv('training_data_features.csv')
training_data_labels = pd.read_csv('training_data_labels.csv')

# Merge training features and labels on 'city', 'year', 'weekofyear'
train_data = training_data_features.merge(training_data_labels, on=['city', 'year', 'weekofyear'])

# Drop unnecessary columns and separate the target variable
X = train_data.drop(columns=['total_cases', 'week_start_date'])
y = train_data['total_cases']

# Convert categorical variable 'city' to numerical using one-hot encoding
X = pd.get_dummies(X, columns=['city'], drop_first=True)

# Handle missing values by filling NaNs with the mean for numeric columns only
X.fillna(X.select_dtypes(include=[np.number]).mean(), inplace=True)
test_data_features.fillna(test_data_features.select_dtypes(include=[np.number]).mean(), inplace=True)

# Apply the same dummy encoding for test data and drop 'week_start_date' as it’s not needed
test_data = pd.get_dummies(test_data_features.drop(columns=['week_start_date']), columns=['city'], drop_first=True)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_data_scaled = scaler.transform(test_data)

# Split the training data for evaluation
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model using MAE on validation set
y_val_pred = model.predict(X_val)
mae_val = mean_absolute_error(y_val, y_val_pred)
print(f"Validation Mean Absolute Error: {mae_val}")

# Predict on the test data
test_predictions = model.predict(test_data_scaled)

# Convert predictions to integer values as required by the submission format
test_predictions = np.round(test_predictions).astype(int)

# Prepare the output DataFrame in the required format
submission = test_data_features[['city', 'year', 'weekofyear']].copy()
submission['total_cases'] = test_predictions

# Display the first few rows of the submission format
submission.head()

# Save the submission DataFrame to a CSV file for submission
submission.to_csv('dengai_submission.csv', index=False)



Validation Mean Absolute Error: 24.065185362129128
